R1CS to QAP
=============

In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Example

In [80]:
# ref: https://medium.com/@VitalikButerin/quadratic-arithmetic-programs-from-zero-to-hero-f6d558cea649

from functools import partial
from klefki.zkp.r1cs import R1CS, mul
from klefki.zkp.qap import R1CS2QAP, proof, verify, transfer
from klefki.curves.barreto_naehrig.bn128 import BN128FP as F
import ast



In [47]:
# map int to field
ciphers = [1,2,3,4,5,6,7,8,9]
times = 5

@R1CS.r1cs(F)
def f(x, k, c):
    y = x + c + k
    return y ** 3

@R1CS.r1cs(F, globals())
def mimc(x, k):
    for i in range(times):
        c = ciphers[i]
        x = f(x, k, c)
    return x + k

In [48]:
f.flatcode

[['+', 'Sym::0', 'x', 'c'],
 ['+', 'y', 'Sym::0', 'k'],
 ['*', 'Sym::1', 'y', 'y'],
 ['*', '~out', 'Sym::1', 'y']]

In [49]:
mimc.flatcode

[['set', 'c', 1],
 ['+', 'Local<Rc(0)>Sym::0', 0, 2],
 ['+', 'Local<Rc(0)>y', 'Local<Rc(0)>Sym::0', 1],
 ['*', 'Local<Rc(0)>Sym::1', 'Local<Rc(0)>y', 'Local<Rc(0)>y'],
 ['*', 'x::0', 'Local<Rc(0)>Sym::1', 'Local<Rc(0)>y'],
 ['set', 'c::1', 2],
 ['+', 'Local<Rc(1)>Sym::0', 0, 2],
 ['+', 'Local<Rc(1)>y', 'Local<Rc(1)>Sym::0', 1],
 ['*', 'Local<Rc(1)>Sym::1', 'Local<Rc(1)>y', 'Local<Rc(1)>y'],
 ['*', 'x::2', 'Local<Rc(1)>Sym::1', 'Local<Rc(1)>y'],
 ['set', 'c::3', 3],
 ['+', 'Local<Rc(2)>Sym::0', 0, 2],
 ['+', 'Local<Rc(2)>y', 'Local<Rc(2)>Sym::0', 1],
 ['*', 'Local<Rc(2)>Sym::1', 'Local<Rc(2)>y', 'Local<Rc(2)>y'],
 ['*', 'x::4', 'Local<Rc(2)>Sym::1', 'Local<Rc(2)>y'],
 ['set', 'c::5', 4],
 ['+', 'Local<Rc(3)>Sym::0', 0, 2],
 ['+', 'Local<Rc(3)>y', 'Local<Rc(3)>Sym::0', 1],
 ['*', 'Local<Rc(3)>Sym::1', 'Local<Rc(3)>y', 'Local<Rc(3)>y'],
 ['*', 'x::6', 'Local<Rc(3)>Sym::1', 'Local<Rc(3)>y'],
 ['set', 'c::7', 5],
 ['+', 'Local<Rc(4)>Sym::0', 0, 2],
 ['+', 'Local<Rc(4)>y', 'Local<Rc(4)>Sym::

The format of a flatcode line is:

$$
\left[Op, Out, S_a, S_b\right]
$$

In [50]:
mimc.var

['~one',
 'x',
 'k',
 '~out',
 'c',
 'Local<Rc(0)>Sym::0',
 'Local<Rc(0)>y',
 'Local<Rc(0)>Sym::1',
 'x::0',
 'c::1',
 'Local<Rc(1)>Sym::0',
 'Local<Rc(1)>y',
 'Local<Rc(1)>Sym::1',
 'x::2',
 'c::3',
 'Local<Rc(2)>Sym::0',
 'Local<Rc(2)>y',
 'Local<Rc(2)>Sym::1',
 'x::4',
 'c::5',
 'Local<Rc(3)>Sym::0',
 'Local<Rc(3)>y',
 'Local<Rc(3)>Sym::1',
 'x::6',
 'c::7',
 'Local<Rc(4)>Sym::0',
 'Local<Rc(4)>y',
 'Local<Rc(4)>Sym::1',
 'x::8']

The format of variable is

$$
[One, Input_0, \cdots, Input_n, Output, S_0, S_1, \cdots, S_n]
$$


In [51]:
assert len(mimc.A[0]) == len(mimc.var)

For each line of flatcodes, we have $A_i.s \circ B_i.s == C_i.s$

In [52]:
s = mimc.witness(F(2))
s

[BN128FP::1,
 BN128FP::2,
 BN128FP::0,
 BN128FP::27,
 BN128FP::1,
 BN128FP::2,
 BN128FP::3,
 BN128FP::9,
 BN128FP::27,
 BN128FP::2,
 BN128FP::2,
 BN128FP::3,
 BN128FP::9,
 BN128FP::27,
 BN128FP::3,
 BN128FP::2,
 BN128FP::3,
 BN128FP::9,
 BN128FP::27,
 BN128FP::4,
 BN128FP::2,
 BN128FP::3,
 BN128FP::9,
 BN128FP::27,
 BN128FP::5,
 BN128FP::2,
 BN128FP::3,
 BN128FP::9,
 BN128FP::27]

In [53]:
sum(mul(mimc.A[0], s)) * sum(mul(mimc.B[0], s)) == sum(mul(mimc.C[0], s))

True

## Gen QAP


In [79]:
r1cs = mimc.r1cs
r1cs

([[BN128FP::21888242871839275222246405745257275088696311157297823662689037894645226208582,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::1,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0],
  [BN128FP::2,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0,
   BN128FP::0],
  [BN128FP::1,
   BN128FP::0,
   BN128FP::0,
   BN12

In [88]:
transfer(mimc.A, field=F)(F(1))

[BN128FP::21888242871839275222246405745257275088696311157297823662689037894645226208582,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0,
 BN128FP::1,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0,
 BN128FP::0]

In [75]:
QAP = R1CS2QAP(*r1cs, F(1221313133), field=F)
A, B, C, Z = QAP

In [76]:
s, H = proof(f.witness(3), A, B, C, Z, field=F)

In [89]:
verify(s, A, B, C, Z, H)

True

In [90]:
A, B, C, Z, H

([BN128FP::18528821486757629599575126182510756786192108733748884350617482518246045257730,
  BN128FP::0,
  BN128FP::14188248263079370240645723444299543202660091185185310248649058977010906106422,
  BN128FP::0,
  BN128FP::8681558859779145740275106502548175731561527437088835768720380000710465944371,
  BN128FP::20472050513588150639542828464324160505322019997805346400832200191089956834344,
  BN128FP::11869158179694763358623636777949118472557743373194323327494525788449775400252,
  BN128FP::5447307428747619787087466600692170657637396311637951189233162325420422642841,
  BN128FP::0,
  BN128FP::3906293114752928409685621749837118442720883203630785933266853976266281547055,
  BN128FP::3364532530325570828090821320598582367047771497497965489506388077495546352120,
  BN128FP::395334118315584227542372297484750615449194963392731222527921901824393641027,
  BN128FP::5163840133312747242231859578442397851741147348134845627848719746686526000913,
  BN128FP::0,
  BN128FP::2661765331673208121755424618390768272366

## PGHR13

In [95]:
from klefki.curves.barreto_naehrig import bn128

ECG = bn128.ECGBN128
G1 = bn128.ECGBN128.G1
G2 = bn128.ECGBN128.G2
G = G1
e = bn128.ECGBN128.e


In [116]:
ECG.lift_x(G2.x)

AttributeError: 'BN128FP2' object has no attribute 'P'